# Ejercicio integrador / Mineria de Datos / Semana 05

<h4><strong>Estudiante:</strong> Pablo Jusim</h4>

<strong>Negocio:</strong> Empresa de análisis financiero.  

<strong>DataSet:</strong> Información de clientes bancarios.  

<strong>Objetivo:</strong> es preparar los datos para un modelo de predicción de solvencia financiera.

### Importaciones

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

### Carga (creación) de data set

In [2]:
df = pd.DataFrame({
"Cliente": [1, 2, 3, 4, 5, 6, 7, 8],
"Edad": [25, 40, 35, 50, 30, 45, 38, 29],
"Ingresos": [50000, 60000, 70000, 80000, 45000, 85000, 72000, 48000],
"Historial_Pagos": ["Bueno", "Regular", "Malo", "Bueno", "Regular", "Malo", "Bueno", "Regular"],
"Categoría": ["A", "B", "C", "A", "B", "C", "A", "B"]
})
df.head()

,Cliente,Edad,Ingresos,Historial_Pagos,Categoría
0,1,25,50000,Bueno,A
1,2,40,60000,Regular,B
2,3,35,70000,Malo,C
3,4,50,80000,Bueno,A
4,5,30,45000,Regular,B


### PreProcesamiento de datos

#### Limpieza de datos

##### Datos nulos

In [6]:
# Ver cantidad de valores nulos por columna
print("Valores nulos por columna:\n", df.isnull().sum())

Valores nulos por columna:
 Cliente            0
Edad               0
Ingresos           0
Historial_Pagos    0
Categoría          0
dtype: int64


In [8]:
# Reemplazo de valores nulos por la media de la columna "Edad"
mode_value = df["Edad"].median()
df.loc[:, "Edad"] = df["Edad"].fillna(mode_value)
# Estas líneas no harán nada ya que no hay valores nulos en el DataFrame

##### Outliyers

In [ ]:
# 1️ Cálculo del IQR de ingresos
Q1 = df["Ingresos"].quantile(0.25)
Q3 = df["Ingresos"].quantile(0.75)
IQR = Q3 - Q1

# 2️ Definir límites para detectar outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# 3️ Filtrar valores dentro de los límites
df_cleaned = df[(df["Ingresos"] >= limite_inferior) & (df["Ingresos"] <= limite_superior)]
print(f'Datos eliminados: {df.size - df_cleaned.size}')

# No se encontraron outliers en la columna "Ingresos" en este caso,
# ya que todos los valores están dentro de los límites definidos por el IQR.

Datos eliminados: 0


#### Transformación de datos numéricos

##### Normalizar Ingresos

In [12]:
scaler = MinMaxScaler()
df_cleaned["Ingreso_normalizado"] = scaler.fit_transform(df_cleaned[["Ingresos"]])

##### Estandarizar edad

In [13]:
# Estandarizar Edad con Z-score Scaling
scaler = StandardScaler()
df_cleaned["Edad_estandarizada"] = scaler.fit_transform(df_cleaned[["Edad"]])
df_cleaned.head()

,Cliente,Edad,Ingresos,Historial_Pagos,Categoría,Ingreso_normalizado,Edad_estandarizada
0,1,25,50000,Bueno,A,0.125,-1.451747
1,2,40,60000,Regular,B,0.375,0.441836
2,3,35,70000,Malo,C,0.625,-0.189358
3,4,50,80000,Bueno,A,0.875,1.704225
4,5,30,45000,Regular,B,0.000,-0.820553


#### Transformación de datos categóricos

##### Aplicar One-Hot Encoding a la columna Historial de Pagos.

In [14]:
df_encod = pd.get_dummies(df_cleaned, columns=["Historial_Pagos"], prefix="H_pagos")

##### Aplicar Label Encoding a la columna Categoría.

In [15]:
# Aplicar Label Encoding a la columna Categoría
le = LabelEncoder()
df_encod["Categoría"] = le.fit_transform(df_encod["Categoría"])
df_encod.head()

,Cliente,Edad,Ingresos,Categoría,Ingreso_normalizado,Edad_estandarizada,H_pagos_Bueno,H_pagos_Malo,H_pagos_Regular
0,1,25,50000,0,0.125,-1.451747,True,False,False
1,2,40,60000,1,0.375,0.441836,False,False,True
2,3,35,70000,2,0.625,-0.189358,False,True,False
3,4,50,80000,0,0.875,1.704225,True,False,False
4,5,30,45000,1,0.000,-0.820553,False,False,True


## 4️. Interacción con una Herramienta de IA (LLM)